In [22]:
import numpy as np
import scipy as sp
import copy
import time
import queue as Q

In [27]:
class widgetState():
    def __init__(self, path, remainingWidget):
        self.remainingWidget = remainingWidget
        self.path = path
        self.frontList = []
        self.completeList = ['A', 'B', 'C', 'D', 'E']
        for i in range(len(remainingWidget)):
            if remainingWidget[i]:
                self.frontList.append(remainingWidget[i][0])
        self.heuristic = 0
        
    def __eq__(self, widget):
        return self.heuristic == widget.heuristic
    
    def __lt__(self, widget):
        return self.heuristic < widget.heuristic

#Pick Widget
    def popComp(self, component):
        widgets = copy.deepcopy(self.remainingWidget)
        for widg in widgets:
            if widg and widg[0] == component:
                widg.pop(0)
        return widgets
                
    def adjacent_minStops(self):
        adjList = []
        lastStop = ''
        if self.path:
            lastStop = self.path[-1]
        for node in set(self.frontList):
            if node != lastStop:
                path = copy.deepcopy(self.path)
                path.append(node)
                pickWidgets = self.popComp(node)
                adjList.append(widgetState(path, pickWidgets))
        return adjList
    
    def adjacent_minMiles(self):
        adjList = []
        lastStop = ''
        if self.path:
            lastStop = self.path[-1]
        for node in self.completeList:
            if node != lastStop:
                path = copy.deepcopy(self.path)
                path.append(node)
            # if keep widgets!!!!!!!!
                keepWidgets = copy.deepcopy(self.remainingWidget)
                adjList.append(widgetState(path, keepWidgets))
            # if pick widgets!!!!!!!!
                if node in self.frontList:
                    pickWidgets = self.popComp(node)
                    adjList.append(widgetState(path, pickWidgets))
        return adjList
        
    def printState(self):
        print("widgets:")
        print(self.remainingWidget)
        print("path:")
        print(self.path)
        print("heuristic:")
        print(self.heuristic)

In [28]:
class widgetPlanning():
    
    def __init__(self):
        self.widgetMat = [
            ['A', 'E', 'D', 'C', 'A'],
            ['B', 'E', 'A', 'C', 'D'],
            ['B', 'A', 'B', 'C', 'E'],
            ['D', 'A', 'D', 'B', 'D'],
            ['B', 'E', 'C', 'B', 'D']]
        self.distanceMat = [
            [0, 1064, 673, 1401, 277],
            [1064, 0, 958, 1934, 337],
            [673, 958, 0, 1001, 399],
            [1401, 1934, 1001, 0, 387],
            [277, 337, 399, 387, 0]]
        self.distMatDic = {'A':0, 'B':1, 'C':2, 'D':3, 'E':4}
#         self.solution = ['B', 'E', 'D', 'A', 'E', 'D', 'C', 'B', 'C', 'A', 'E', 'D']
        self.solution = []
    
    def calculateMiles(self, path):
        miles = 0
        for i in range(len(path)-1):
            miles += self.distanceMat[self.distMatDic[path[i]]][self.distMatDic[path[i+1]]]
        return miles
    
    def maxMiles(self, remainingWidgets):
        maxMiles = 0
        for path in remainingWidgets:
            maxMiles = max(maxMiles, self.calculateMiles(path))
        return maxMiles
    
    def printSolution(self):
        miles = self.calculateMiles(self.solution)
        print("Route:" + str(self.solution))
        print("Miles:" + str(miles))

    def AStarPlanning_minStops(self):
        queue = Q.PriorityQueue()
        path = []
        remainingWidget = self.widgetMat
        start = widgetState(path, remainingWidget)
        start.heuristic = len(start.path)+len(max(start.remainingWidget,key=len))
        queue.put((start.heuristic, start))
        while not queue.empty():
            current = queue.get()[1]
#            current.printState()
            empty = True
            for i in current.remainingWidget:
                if i:
                    empty = False
            if not empty:
#                 print("haha")
                for node in current.adjacent_minStops():
#                     node.printState()
                    node.heuristic = len(node.path)+len(max(node.remainingWidget,key=len))
                    queue.put((node.heuristic, node))
            else: 
                self.solution = current.path
                self.printSolution()
                return
            
    def AStarPlanning_minMiles(self):
        queue = Q.PriorityQueue()
        path = []
        remainingWidget = self.widgetMat
        start = widgetState(path, remainingWidget)
        start.heuristic = self.calculateMiles(start.path)+self.maxMiles(start.remainingWidget)
        queue.put((start.heuristic, start))
        while not queue.empty():
            current = queue.get()[1]
#             current.printState()
            empty = True
            for i in current.remainingWidget:
                if i:
                    empty = False
            if not empty:
#                 print("haha")
                for node in current.adjacent_minMiles():
                    node.heuristic = self.calculateMiles(node.path)+self.maxMiles(node.remainingWidget)
#                     node.printState()
                    queue.put((node.heuristic, node))
            else: 
                self.solution = current.path
                self.printSolution()
                return

In [29]:
widget = widgetPlanning()
widget.AStarPlanning_minStops()
widget.AStarPlanning_minMiles()

Route:['A', 'B', 'E', 'D', 'C', 'A', 'D', 'B', 'C', 'D', 'E']
Miles:9143
Route:['D', 'E', 'A', 'E', 'B', 'E', 'D', 'E', 'C', 'A', 'E', 'B', 'E', 'C', 'E', 'D']
Miles:5597
